In [2]:
import sys
sys.path.append("../") 

In [3]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import max_error, mean_squared_error, median_absolute_error, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from modelos import *

In [4]:
def calcular_metricas(y_true, y_predict):
    return {
        "MAX_ERROR": max_error(y_true, y_predict),
        "MEAN_SQ_ERROR": mean_squared_error(y_true, y_predict),
        "MEDIAN_ABS_ERROR": median_absolute_error(y_true, y_predict),
        "MEAN_ABS_ERROR": mean_absolute_error(y_true, y_predict),
    }

In [5]:
def heuristica(Vviento):
    """Definimos la heurística basada en la Escala de Beaufort"""
    if Vviento < 3:
        return 0.4
    elif Vviento < 7:
        return 0.6
    elif Vviento < 12:
        return 1
    elif Vviento < 20:
        return 1.5
    elif Vviento < 26:
        return 2
    elif Vviento < 36:
        return 2.5
    elif Vviento < 46:
        return 3.2
    elif Vviento < 56:
        return 4.2
    elif Vviento < 66:
        return 5.4
    elif Vviento < 76:
        return 6.5
    elif Vviento < 88:
        return 8
    elif Vviento < 101:
        return 10
    elif Vviento >= 101:
        return 12

## Datos nuevos

Cargamos los datos que hemos capturado hasta ahora

In [6]:
data = pd.read_csv("./clean/forecast_merged.csv")
data = data.dropna() #Quitamos todas aquellas filas que tengan algun na

In [7]:
X = data.drop(columns = "AlturaOlas")
X = X.drop(columns=["antelacion", 'anio', 'mes', 'dia', 'hora'])
y = data["AlturaOlas"]

Obtenemos la escala que hemos utilizado para entrenar el modelo

In [8]:
X_model = X.copy()
scaler = get_scaler()
X_model = scaler.transform(X_model)

Cargamos el modelo para poder comparar con la heuristica

In [9]:
with open("../modelos/modelo_definitivo/modelo_rf_def.pkl", 'rb') as file:
    model = pickle.load(file)
model

RandomForestRegressor(max_depth=28, min_samples_leaf=4, min_samples_split=4,
                      n_estimators=201, random_state=777)

Realizamos las predicciones

In [10]:
predicciones_modelo = model.predict(X_model)
predicciones_heuristica = X["Vviento"].apply(heuristica)

C:\Users\Mario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Analizamos los resultados

In [11]:
calcular_metricas(y, predicciones_modelo)

{'MAX_ERROR': 1.1858099030355385,
 'MEAN_SQ_ERROR': 1.0113543909042624,
 'MEDIAN_ABS_ERROR': 1.0633592835886514,
 'MEAN_ABS_ERROR': 0.9831377743656742}

In [12]:
calcular_metricas(y, predicciones_heuristica)

{'MAX_ERROR': 0.6,
 'MEAN_SQ_ERROR': 0.0699526066350711,
 'MEDIAN_ABS_ERROR': 0.09999999999999998,
 'MEAN_ABS_ERROR': 0.19763033175355446}

Como podemos observar, la heurística funciona mejor que le modelo. Esto se puede deber a los pocos datos nuevos de los que disponemos, por lo que veamos si con los datos anteriores sigue funcionando mejor la heuristica o no

## Datos históricos

In [13]:
X, y = datos_full()
X = X.drop(columns=["Temperatura", 'wspd', 'anio', 'mes', 'dia', 'hora'])
X_model = X.copy()

scaler = get_scaler()
X_model = scaler.transform(X_model);
predicciones_modelo = model.predict(X_model)
predicciones_heuristica = X["Vviento"].apply(heuristica)

C:\Users\Mario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [14]:
calcular_metricas(y, predicciones_modelo)

{'MAX_ERROR': 5.368412524457302,
 'MEAN_SQ_ERROR': 0.9746322071373777,
 'MEDIAN_ABS_ERROR': 0.8253401407879015,
 'MEAN_ABS_ERROR': 0.8309490032976403}

In [15]:
calcular_metricas(y, predicciones_heuristica)

{'MAX_ERROR': 4.4,
 'MEAN_SQ_ERROR': 0.48961046057767366,
 'MEDIAN_ABS_ERROR': 0.3999999999999999,
 'MEAN_ABS_ERROR': 0.5072209211553473}

Sorprendentemente, la heurística sigue funcionando mejor